> This notebook demonstrates a generative climate model output emulator based on score-based diffusion models. The version shown here allows to run emulators of monthly averaged near-surface temperature, precipitation, relative humidity, wind speed for MPI-ESM1-2-LR. Full description of the methodology and evaluation are provided in the preprint.

### Setup collab on GPU

- From the dropdown menu, select Change runtime type.
<br>
<br>
<img src="https://github.com/shahineb/climemu/blob/dev/examples/examples/docs/1.jpg?raw=1" width="400" />

<br>

- In the Hardware accelerator section, select T4 GPU and click Save
<br>
<br>
<img src="https://github.com/shahineb/climemu/blob/dev/examples/examples/docs/2.jpg?raw=1" width="300" />

<br>

You can now start the session by running the next cell.

In [2]:
# Install package with GPU support
%%capture
!pip install climemu[cuda12] cartopy

In [3]:
# Import used dependencies
import climemu
import numpy as np
import xarray as xr
from tqdm.notebook import tqdm
import utils

In [4]:
# Instantiate emulator
emulator = climemu.build_emulator("MPI-ESM1-2-LR")

# Download pretrained weights and compile emulator (~1min)
emulator.load()
emulator.compile(n_samples=3)   # Nb of samples generated at each function call

In [20]:
# Choose GMST and month at which we emulate
gmst = 2       # GMST anomaly wrt piControl (°C)
month = 3      # Month index (1-12)

# Fix random seed for reproducibility
np.random.seed(42)

# Call the emulator e.g. 10 times
n_call = 10
samples = []
for _ in tqdm(range(n_call), desc=f"Emulating for ΔΤ = {gmst:.2f} °C for month {month}"):
  sample = emulator(gmst, month, xarray=True)
  samples.append(sample)

Emulating for ΔΤ = 2.00 °C for month 3:   0%|          | 0/10 [00:00<?, ?it/s]

In [22]:
# Combine to form 3*10 = 30 emulated ensemble members
combined = xr.concat(samples, dim="member")
combined = combined.assign_coords(member=range(1, combined.sizes["member"] + 1))
combined

<xarray.Dataset> Size: 9MB
Dimensions:  (member: 30, lat: 96, lon: 192)
Coordinates:
  * lat      (lat) float64 768B -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
  * lon      (lon) float64 2kB 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
  * member   (member) int64 240B 1 2 3 4 5 6 7 8 9 ... 23 24 25 26 27 28 29 30
Data variables:
    tas      (member, lat, lon) float32 2MB ...
    pr       (member, lat, lon) float32 2MB ...
    hurs     (member, lat, lon) float32 2MB ...
    sfcWind  (member, lat, lon) float32 2MB ...

In [ ]:
# Do your plots
